### **This notebook perform automatical extraction of the impervious cover of each Landsat image from 1986-2020**
##### Step 1: Obtain the Landsat image data (tfrecord) for impervious prediction.
##### Step 2: Prediction with the pretrained model
##### Step 3: Write the result into Google Cloud Storage and copy the corresponding .json file from google drive to google cloud storage.
##### Step 4: Upload the result (tfrecord) and .json files to the Earth Engine.

In [27]:
## Mount on google drive
from google.colab import drive
drive.mount('/content/drive/')
# Cloud authentication.
from google.colab import auth
auth.authenticate_user()


Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


### **We use Tensorflow 2.2.0 here!**

In [28]:
# !pip install tensorflow==2.2.0
import tensorflow as tf
print(tf.__version__)

2.3.0


In [29]:

import os
os.chdir("/content/drive/My Drive/Earth-Engine-with-Deep-Learning")
import time
import json
from utils import imgShow

### **Note: the pretrained model is trained with tensorflow 2.2.0.**

In [33]:
## Super-parameter: 
## Data (Google Drive)---> Result(Google Cloud Storage) ---> Result (Earth Engine)
####################################################
##  Google Cloud Storage: path for result writer 
Project = 'my-project-20200813'
Bucket = 'earth-engine-bucket-1'
result_gs_dir = 'gs://' + Bucket + '/prediction_impervious_sz/'

#### Google Drive: the pretrained model and collected Landsat image (tfrecord).
model_path = '/content/drive/My Drive/Earth-Engine-with-Deep-Learning/models/pretrain/unet_MSMT_train_50epoch_CEloss_nosiyAug/model'
data_folder = '/content/drive/My Drive/EE_Image/Landsat_image_sz'
# Landsat image (.tfrecord) list and the corresponding .json file list.
os.chdir(data_folder)
filesList = !ls -1
FilesList_img = [s for s in filesList if '.tfrecord.gz' in s]
FilesList_json = [s for s in filesList if '.json' in s]
#### load the pre-trained model
model_pretrain = tf.keras.models.load_model(model_path)

#### Earth Engine: the path for result (.TFRecord and .json) uploading 
ee_output_folder = 'users/xin_luo/Impervious_Mapping_SZ/result'

## Data features
bands_l5 = bands_l7 = ['B1', 'B2', 'B3', 'B4', 'B5', 'B7']
bands_l8 = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7']  # determine the bands to be collected
## Patch size: Kernel_size + Kernel_buffer
kernel_size = [208, 208]
kernel_buffer = [48, 48]   # left buffer + right buffer = 48, the same to up and down
buffered_shape = [kernel_size[0]+kernel_buffer[0], kernel_size[1]+kernel_buffer[1]]


In [ ]:
date_years = ['1986','1987','1988','1989','1990','1991','1992','1993','1994',\
              '1995','1996','1997','1998','1999','2000','2001','2002','2003',\
              '2004','2005','2006','2007','2008','2009','2010','2011','2012',\
              '2013','2014','2015','2016','2017','2018','2019','2020']
date_years_l5 = ['1986','1987','1988','1989','1990','1991','1992','1993',\
                 '1994','1995','1996','1997','1998','1999','2000','2001',\
                 '2002','2003','2004','2005','2006','2007','2008','2009',\
                 '2010','2011']
date_years_l7 = ['2012']
date_years_l8 = ['2013','2014','2015','2016','2017','2018','2019','2020']


In [ ]:
## the selected bands corresponding to Landsat image (Landsat 5,7,and8). 
def getBands(date_year):
    '''
    input: the date(year) of the landsat image
    ouput: the and bands selection 
    '''
    if date_year in date_years_l5+date_years_l7:
        bands = bands_l5
    elif date_year in date_years_l8:
        bands = bands_l8
    return bands

## functions used to parse the .tfrecord file
def parse_image(example_proto):
    return tf.io.parse_single_example(example_proto, featuresDict)

def toTensor(inputs):
    inputsList = [inputs.get(key) for key in bands]
    stacked = tf.stack(inputsList, axis=0)
    stacked = tf.transpose(stacked, [1, 2, 0])
    return stacked


In [ ]:
for date_year in date_years:
    print(date_year)
    os.chdir(data_folder)
    ### 1.Super parameters
    ## define feature dictionary
    bands = getBands(date_year)
    imageColumns = [tf.io.FixedLenFeature(shape=buffered_shape, dtype=tf.float32) 
                    for k in bands]
    featuresDict = dict(zip(bands, imageColumns))    # 用于tfrecord文件中数据描述的字典
    ## .TFRecord image and the .json file.
    landsat_img = [img for img in FilesList_img if date_year in img]
    landsat_json = [json for json in FilesList_json if date_year in json][0]
    ## obtain number of patches with the .json file 
    jsonText = !cat {landsat_json}
    mixer = json.loads(jsonText.nlstr)
    patches = mixer['totalPatches']
    ## Google Cloud Storage directory to write prediction result 
    result_gs_path = result_gs_dir + 'Imper_' + date_year + '.TFRecord'
    ### 2.Create a dataset from the TFRecord file(s) in Cloud Storage.  
    
    imageDataset = tf.data.TFRecordDataset(landsat_img, compression_type='GZIP')
    imageDataset = imageDataset.map(parse_image, num_parallel_calls=5)
    imageDataset = imageDataset.map(toTensor).batch(1)
    ### 3.Prediction
    predictions = model_pretrain.predict(imageDataset, steps=patches, verbose=1)
    ### 4.Write the prediction result to Google Cloud Storage
    writer = tf.io.TFRecordWriter(result_gs_path)
    i_patch = 0
    for predPatch in predictions:
        # print('Writing patch ' + str(i_patch) + '...')
        predPatch = predPatch[      # clip the overlay region
            kernel_buffer[0]//2:kernel_buffer[0]//2+kernel_size[0], 
            kernel_buffer[1]//2:kernel_buffer[1]//2+kernel_size[1]
            ]
        # Creat an example
        example = tf.train.Example(
        features=tf.train.Features(
            feature={
            'impervious': tf.train.Feature(
                float_list=tf.train.FloatList(
                    value=predPatch.flatten()))
            }
        )
        )
        # Write the example.
        writer.write(example.SerializeToString())
        i_patch += 1
    writer.close()
    ### 5.Upload .json file from google drive to google storage
    !gsutil -m cp -r {landsat_json} {result_gs_dir}


In [ ]:
## Authenticate to Earth Engine
import ee
ee.Authenticate()
ee.Initialize()

In [36]:
# !pip install oauth2client==3.0.0
# upload both the .TFRecord file and .json file from google storage to the earth engine.
for date_year in date_years:
    print(date_year)
    ## result (.TFRecord, .json) path in the Google Cloud Storage
    result_gs_path = result_gs_dir + 'Imper_' + date_year + '.TFRecord'
    landsat_json = [json for json in FilesList_json if date_year in json][0]
    jsonFile_gs_path = 'gs://' + Bucket+ '/prediction_impervious_sz/' + landsat_json
    # upload path of the Earth Engine
    ee_output_image = ee_output_folder + '/' + 'Imper_' + date_year
    ## upload
    !earthengine upload image --asset_id={ee_output_image} {result_gs_path} {jsonFile_gs_path}


1986
Instructions for updating:
non-resource variables are not supported in the long term
Running command using Cloud API.  Set --no-use_cloud_api to go back to using the API

I0925 06:15:26.763422 140314904721280 discovery.py:894] URL being requested: GET https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/algorithms?prettyPrint=false&alt=json
I0925 06:15:28.635452 140314904721280 discovery.py:894] URL being requested: POST https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/image:import?alt=json
Started upload task with ID: 2NVGB64K4R3HKVGWKSV7OYMV
